# Agenda

0. Q&A
1. `**kwargs` and other parameter types
2. Scoping (LEGB)
3. Enclosing functions
4. Dispatch tables
5. Comprehensions
6. Sorting and `lambda` (and passing functions as arguments to other functions)
7. Type hints/annotations

In [1]:
d = {}
d['a'] = 10
hash('a') % 8

0

In [2]:
hash('b') % 8

2

In [3]:
hash('c') % 8

6

In [4]:
hash('d') % 8

3

In [5]:
for one_letter in 'abcdefghij':
    print(f'{one_letter}: {hash(one_letter) % 8}')

a: 0
b: 2
c: 6
d: 3
e: 0
f: 5
g: 6
h: 0
i: 6
j: 4


In [6]:
d['e'] = 500

In [7]:
d = {'a':10, 'b':20, 'c':30}

new_stuff = {'b':400, 'c':900, 'd':1600}

d.update(new_stuff)   # this modifies d, the "receiving" dict, to take all key-value pairs from new_stuff

d

{'a': 10, 'b': 400, 'c': 900, 'd': 1600}

In [8]:
d = {'a':10, 'b':20, 'c':30}
new_stuff = {'b':400, 'c':900, 'd':1600}

# we can, as of 3.10 (?), use the | operator 

d | new_stuff   # this returns a new dict based merging d + new_stuff together, but doesn't modify d or new_stuff

{'a': 10, 'b': 400, 'c': 900, 'd': 1600}

In [9]:
d

{'a': 10, 'b': 20, 'c': 30}

In [10]:
# if you do want to modify d, you can use |=

d |= new_stuff    # this is (I think) identical in behavior to dict.update

In [11]:
d

{'a': 10, 'b': 400, 'c': 900, 'd': 1600}

# Parameter types for functions

1. Mandatory parameters (positional or keyword arguments)
2. Optional parameters (positional or keyword arguments), with a default argument value stored in the function's `__defaults__` tuple
3. `*args`, where `args` is a tuple containing all of the positional arguments that no other parameter got

In [12]:
def add(first, second):
    return first + second

In [13]:
t = (10, 2)

add(t)  # will this work?  No... we're passing 1 argument, but add requires 2

TypeError: add() missing 1 required positional argument: 'second'

In [14]:
# we saw that we can "unroll" the elements of t:

add(*t)   # this turns the 2-element tuple into two separate arguments

12

# Keyword arguments

We saw that we can call a function with one or more sets of "keyword arguments," where it looks like `NAME=VALUE`. All keyword arguments must come after all positional arguments.

`**kwargs` is the keyword-argument analog to `*args`, which only works with positional arguments. `kwargs` is a dict in which the keys are strings, the names of the keyword arguments that were passed, and the values are whatever values were associated with them. It contains all of the keyword arguments that no other variable accepted.

In [15]:
def myfunc(a, b, **kwargs):
    return f'{a=}, {b=}, {kwargs=}'

In [16]:
myfunc(10, 20)  # just passing two positional arguments

'a=10, b=20, kwargs={}'

In [18]:
myfunc(a=10, b=20)  # known parameters get the values of the keyword arguments

'a=10, b=20, kwargs={}'

In [19]:
myfunc(a=10, b=20, c=30)

"a=10, b=20, kwargs={'c': 30}"

In [20]:
myfunc(a=10, b=20, c=30, d=40, e='hello', f={'x':1, 'y':2})

"a=10, b=20, kwargs={'c': 30, 'd': 40, 'e': 'hello', 'f': {'x': 1, 'y': 2}}"

# Why do we need `**kwargs`?

I can think of two reasons:

1. We have a function that takes *so* many arguments that the signature is unreadable. Instead, we can just accept `**kwargs`, and have the user pass whichever of the arguments they find interesting/necessary. We have to look in the dict to see what keys were passed, but the function signature is still more readable.
2. You have a function that knows what to do, but doesn't know what parameter names or values it'll get. This is typically/often for formatting purposes.

You can have `**kwargs` in your function after `*args` (if it exists). 

In [21]:
# what if I do this:

# parameters:   a    b    kwargs
# arguments     10   20    {'c':30}

myfunc(10, 20, c=30, b=200)

TypeError: myfunc() got multiple values for argument 'b'

# Parameter types for functions

1. Mandatory parameters (positional or keyword arguments)
2. Optional parameters (positional or keyword arguments), with a default argument value stored in the function's `__defaults__` tuple
3. `*args`, where `args` is a tuple containing all of the positional arguments that no other parameter got
4. `**kwargs`, where `kwargs` is a dict containing all of the keyword arguments that no other parameter got

In [22]:
def myfunc(a, b=10, *args, **kwargs):
    return f'{a=}, {b=}, {args=}, {kwargs=}'

In [23]:
myfunc(3)

'a=3, b=10, args=(), kwargs={}'

In [24]:
myfunc(3, x=100, y=200, z=300)

"a=3, b=10, args=(), kwargs={'x': 100, 'y': 200, 'z': 300}"

In [25]:
# how can I pass values to args?
# Just pass more positional arguments...

myfunc(3,4,5,6,7, x=100, y=200)

"a=3, b=4, args=(5, 6, 7), kwargs={'x': 100, 'y': 200}"

In [26]:
# how can I pass values to args
# and *not* overwrite the default value in b?

# you can't.

# but... we'll see an alternative solution soon

# Exercise: XML generator

Just to remind you, XML works based on "tags." Each tag has a name, optional attributes in the open tag, and optional content:

    <tagname></tagname>     # empty
    <name>Reuven</name>     # regular tag with content
    <a><b><c>d</c></b></a>  # nested tags
    <a x="1" y="2">b</a>    # attributes in the opening tag

I want you to write a function, `xml`, that takes 1, 2, or more arguments:

- If we pass a single argument, then that's the name of the tag, and we should return a string with its opening and closing tags, but no content.
- If we pass two arguments, then that's the name of the tag and its content. We should return a string with the tag (opening and closing), and the second argument between them.
- If we pass two arguments plus any keyword arguments, the keyword args are all turned into attributes inside of the opening tag. Note that officially, attributes should have a name, the `=` sign, and then a value inside of double quotes.

Example:

    xml('a')            # '<a></a>'
    xml('a', 'b')       # '<a>b</a>'
    xml('a', 'b', x=1)  # <a x="1">b</a>
    

In [34]:
def xml(tagname, content='', **kwargs):
    if not kwargs:
        print(f'\tNo attributes passed; expect none in the output')
    else:
        print(f'\tGot {len(kwargs)} attributes')
    
    attributes = ''
    for key, value in kwargs.items():
        attributes += f' {key}="{value}"'
    
    output = f'<{tagname}{attributes}>{content}</{tagname}>'
    return output


print(xml('a'))
print(xml('a', 'b'))
print(xml('a', 'b', x=1, y=2))

	No attributes passed; expect none in the output
<a></a>
	No attributes passed; expect none in the output
<a>b</a>
	Got 2 attributes
<a x="1" y="2">b</a>


# Parameter types for functions

1. Mandatory parameters (positional or keyword arguments)
2. Optional parameters (positional or keyword arguments), with a default argument value stored in the function's `__defaults__` tuple
3. `*args`, where `args` is a tuple containing all of the positional arguments that no other parameter got
4. `**kwargs`, where `kwargs` is a dict containing all of the keyword arguments that no other parameter got

In [35]:
def myfunc(a, b=10, *args, **kwargs):
    return f'{a=}, {b=}, {args=}, {kwargs=}'

In [36]:
myfunc(2,4,6,8)

'a=2, b=4, args=(6, 8), kwargs={}'

In [37]:
# if we want to leave be with its default, but allow us at the same time to
# give positional argument values to args, we can move b=10 to *after* the mention of
# *args. Then it becomes a keyword-only parameter, one that can only get values with
# keyword arguments.

def myfunc(a, *args, b=10, **kwargs):
    return f'{a=}, {b=}, {args=}, {kwargs=}'

In [38]:
myfunc(2,4,6,8)

'a=2, b=10, args=(4, 6, 8), kwargs={}'

In [39]:
# in order set b's value we need to explicitly mention it as a keyword argument.

myfunc(2,4,6,8, b=999)

'a=2, b=999, args=(4, 6, 8), kwargs={}'

In [40]:
# what if we put b after *args, but we *don't* give it a default?
# then it becomes a mandatory keyword-only parameter; we must pass a value to it
# as a keyword argument when we invoke the function.

#          pos/kw    pos only      kw only
def myfunc(a,        *args,        b,         **kwargs):   # notice that b no longer has a default
    return f'{a=}, {b=}, {args=}, {kwargs=}'

In [41]:
myfunc(2,4,6,8)

TypeError: myfunc() missing 1 required keyword-only argument: 'b'

In [42]:
myfunc(2,4,6,8, b=999)

'a=2, b=999, args=(4, 6, 8), kwargs={}'

# Parameter types for functions

1. Mandatory parameters (positional or keyword arguments)
2. Optional parameters (positional or keyword arguments), with a default argument value stored in the function's `__defaults__` tuple
3. `*args`, where `args` is a tuple containing all of the positional arguments that no other parameter got, or `*` to indicate the end of the positional parameters
4. Mandatory keyword-only parameters
5. Optional keyword-only parameters (with defaults)
6. `**kwargs`, where `kwargs` is a dict containing all of the keyword arguments that no other parameter got

In [45]:
# what if I want to write a function whose parameters are *all*
# keyword only?

def myfunc(*, a, b):   # if we put * in the list of parameters, that means: after here, only kw args, but no *args
    return a + b

In [46]:
myfunc(a=10, b=20)

30

In [47]:
myfunc(10, 20)

TypeError: myfunc() takes 0 positional arguments but 2 were given

In [48]:
len('abcd')

4

In [49]:
help(len)  # what does this function do, and what is its signature?

Help on built-in function len in module builtins:

len(obj, /)
    Return the number of items in a container.



# / in a function signature

`/` indicates that up to that point, all of the parameters are positional only. You cannot pass them as keyword arguments.



In [50]:
def myfunc(a, b, /):
    return a + b

In [51]:
myfunc(a=10, b=20)

TypeError: myfunc() got some positional-only arguments passed as keyword arguments: 'a, b'

In [52]:
myfunc(10, 20)

30

# Who needs this?

If your function takes both positional arguments *and* `**kwargs`, it's possible that someone will pass a keyword argument that'll get misinterpreted as a positional argument, or at least meant for a names parameter. Using `/` allows us to have a parameter, and assign to it, but also to get keyword arguments with that name.

In [53]:
def write_config(filename, **kwargs):    # let's write the key-value pairs of kwargs into filename, one line at a time
    with open(filename, 'w') as f:
        for key, value in kwargs.items():
            f.write(f'{key}={value}\n')

In [54]:
write_config('config.txt', a=10, b=20, c=30)

In [55]:
!cat config.txt

a=10
b=20
c=30


In [56]:
# but what happens if I do this:

write_config('config.txt', name='Reuven', filename='myfile.txt')

TypeError: write_config() got multiple values for argument 'filename'

In [57]:
# let's rewrite the function such that filename is positional only
# then we won't have that clash!

def write_config(filename, /, **kwargs): 
    with open(filename, 'w') as f:
        for key, value in kwargs.items():
            f.write(f'{key}={value}\n')

In [58]:
write_config('config.txt', 
             name='Reuven', filename='myfile.txt')

In [59]:
!cat config.txt

name=Reuven
filename=myfile.txt


# Parameter types for functions

1. Positional only, followed by `/`
2. Mandatory parameters (positional or keyword arguments)
2. Optional parameters (positional or keyword arguments), with a default argument value stored in the function's `__defaults__` tuple
3. `*args`, where `args` is a tuple containing all of the positional arguments that no other parameter got, or `*` to indicate the end of the positional parameters
4. Mandatory keyword-only parameters
5. Optional keyword-only parameters (with defaults)
6. `**kwargs`, where `kwargs` is a dict containing all of the keyword arguments that no other parameter got

# Return values

- If a function fails to use `return`, then it automatically returns `None` when the function exits.
- If a function uses just `return` with no value after it, then it returns `None`
- A function can otherwise return any value it wants, with no limits or exceptions. This means that we can return a tuple of values, effectively returning more than one value from our function.


In [60]:
# example : a function that takes any number of integers and returns the min and max of them

def min_and_max(*numbers):
    return min(numbers), max(numbers)

In [61]:
min_and_max(10, 20, -5, -7, 23, 1)

(-7, 23)

In [62]:
# I can use unpacking to retrieve them into separate variables
smallest, largest = min_and_max(10, 20, -5, -7, 23, 1)

In [63]:
smallest

-7

In [64]:
largest

23

In [67]:
def myfunc():
    print(2+3)  # this function might print something, but it returns None

In [68]:
myfunc()

5


In [69]:
x = myfunc()

5


In [70]:
type(x)

NoneType

In [71]:
# I can use a type annotation here 
# name is a string, and the function returns a string
def hello(name:str) -> str:
    return f'Hello, {name}!'

In [72]:
hello(5)

'Hello, 5!'

# Scoping

The whole idea of scoping is: When do variables exist, and when do they cease to exist?

This is useful if we reuse the same variable name in multiple programs/functions. Also, if we want access to a variable, is it still around?

At a very basic level, Python only really has two scopes:
- Global (everything outside of a function)
- Local (everything inside of a function)

In [74]:
x = 100

for i in range(10):   # no function body, thus no new scope -- thus, x and i are global variables
    x = i ** 3

x

729

In [77]:
# this prints 100 -- but why?
# we're in the global scope, and Python looks for a global variable x
# where does it look? It says 'x' in globals(), finds it, and retrieves globals()['x']

x = 100

print(f'x = {x}')

x = 100


In [76]:
'x' in globals()

True

In [78]:
# let's add a function to the mix

x = 100

def myfunc():
    print(f'In myfunc, x = {x}')   # is x local? No. is x global? yes!
    
print(f'before, x = {x}')
myfunc()
print(f'after, x = {x}')

before, x = 100
In myfunc, x = 100
after, x = 100


# Scoping search path in Python: LEGB

Python has a total of *four* scopes. It searches through each of them when it needs to find a variable. Actually, it searches through all four when we're in a function, and only in the final two when we're outside of a function:

- `L` -- local, for local variables (this is where we start if we're in a function)
- `E` -- enclosing, if our function was defined inside of another function
- `G` -- global, for global variables (this is where we start if we're outside of a function)
- `B` -- builtin, names that are defined by Python and stick around a while

In [79]:
# what if we assign to x inside of our function?

x = 100

def myfunc():
    x = 200
    print(f'In myfunc, x = {x}') # is x local (i.e., 'x' in __code__.co_varnames)? yes, and its value is 200
    
print(f'before, x = {x}')  # is x global? yes, and its value is 100
myfunc()
print(f'after, x = {x}')   # is x global (still)? yes, 100

before, x = 100
In myfunc, x = 200
after, x = 100


In [80]:
# when did Python notice that x was local inside of myfunc? Not when we ran it, but 
# when we defined it.

myfunc.__code__.co_varnames   # local variable names in myfunc, as recorded at definition/compile time

('x',)

In [81]:
# let's change things a little bit

x = 100

def myfunc():
    print(f'In myfunc, x = {x}') # is x local? yes ... but it has no value
    x = 200   # this, anywhere in the function, forces x to be local
    
print(f'before, x = {x}')  # is x global? yes, 100
myfunc()
print(f'after, x = {x}')  

before, x = 100


UnboundLocalError: cannot access local variable 'x' where it is not associated with a value

In [84]:
def myfunc():
    y += 1    # y was marked as local at compiled time; at runtime, we're saying y = y + 1

myfunc()

UnboundLocalError: cannot access local variable 'y' where it is not associated with a value

In [85]:
myfunc.__code__.co_varnames

('y',)

In [86]:
# what if we want, from inside of our program, to modify the global x?

x = 100

def myfunc():
    global x   # this tells Python's compiler: Don't mark x as local, even if/when we assign to it
    x = 200    # here, we are assigning to the global x
    print(f'In myfunc, x = {x}')
    
print(f'before, x = {x}') 
myfunc()
print(f'after, x = {x}')  

before, x = 100
In myfunc, x = 200
after, x = 200
